<a href="https://colab.research.google.com/github/SeyedMehdi110/Per-1/blob/main/Question3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing required libraries 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import cvxpy as cp
from sklearn.metrics import accuracy_score 

In [3]:
#reading data
dataframe = pd.read_csv('train.csv',header = None)

dataframe[0].replace(0, -1,inplace = True)
y = dataframe[0]
X = dataframe.drop([0], axis=1)

#splitting data
data_train, data_val, label_train, label_val = train_test_split(X, y, test_size=0.42, random_state=1)

#importing test data
test_set = pd.read_csv('test.csv',header = None)
test_set[0].replace(0, -1,inplace = True)

label_test = test_set[0]
data_test = test_set.drop([0], axis=1)


data_train=np.array(data_train)
data_test=np.array(data_test)
data_val=np.array(data_val)

label_train = np.array(label_train)
label_test = np.array(label_test)
label_val = np.array(label_val)


In [4]:
import numpy as np
import cvxopt


class SupportVectorMachine(object):
    def __init__(self, C=None):
        self.C = C

        
    def fit(self, X, y):
        n = X.shape[0]
          
        # Calculate the Gram matrix.
        K = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                K[i,j] = np.dot(X[i], X[j])
        
        # Setup necessary matrices to be used in the calculation of the Lagrangian
        # multipliers.
        P = cvxopt.matrix(np.outer(y,y) * K)        
        q = cvxopt.matrix(-np.ones((n,1)))
        A = cvxopt.matrix(y.reshape(1,-1))
        b = cvxopt.matrix(np.zeros((1,1)))
        
        if self.C is None:
            h = cvxopt.matrix(np.zeros((n, 1)))
            G = cvxopt.matrix(-np.eye(n))
        else:
            G_lower = -np.eye(n)
            G_upper = np.eye(n)
            h_lower = np.zeros((n,1))
            h_upper = np.full((n,1),self.C)
            G = cvxopt.matrix(np.concatenate((G_lower, G_upper), axis=0))
            h = cvxopt.matrix(np.concatenate((h_lower,h_upper), axis=0))
        
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)  
        
        # Extract the Lagrangian multipliers.
        self.alphas = (np.array(solution['x'])).flatten()
        
        self.support_alphas_indices = []
        self.support_alphas = []
        self.support_vectors = []
        self.support_ys = []
        
        # Determine the support vectors, their associated Lagrangian multipliers and labels.
        for i, x in enumerate(self.alphas):
            if x > 1e-4:
                self.support_alphas_indices.append(i)
                self.support_alphas.append(self.alphas[i])
                self.support_vectors.append(X[i])
                self.support_ys.append(y[i])


        print("Sum of Alphas= ",np.sum(self.alphas))  
        self.support_alphas_indices = np.array(self.support_alphas_indices)
        self.support_alphas = np.array(self.support_alphas)
        self.support_vectors = np.array(self.support_vectors)
        self.support_ys = np.array(self.support_ys)
        
        isSV = self.alphas > 1e-5
        
        # Calculate the model intercept.
        self.b = 0
        for idx, i in enumerate(self.support_alphas_indices):
            self.b += self.support_ys[idx]
            kernel_vec = K[i]
            for j,k,l in zip(self.support_alphas, self.support_ys, kernel_vec[isSV]):
                self.b -= (j * k * l)
        self.b /= len(self.support_alphas)
        
        # Calculate the model weights if applicable.
        self.w = np.zeros(X.shape[1])
        for i in range(len(self.support_alphas)):
            multiplier = self.support_alphas[i] * self.support_ys[i]
            for x in range(X.shape[1]):
                self.w[x] +=  multiplier * self.support_vectors[i][x]

            
            
    def decision_function(self,X):
        '''Calculate and return the real valued prediction of the model.'''
        try:
            if self.w is not None:
                return X @ self.w + self.b
            else:
                preds = []
                for x in X:
                    summation = 0
                    for alpha,y,sv in zip(self.support_alphas, self.support_ys, self.support_vectors):
                        summation += alpha * y * self.apply_kernel_function(x,sv)
                    preds.append(summation)
                return preds + self.b   
        except AttributeError:
            print('Model not fitted, call \'fit\' with appropriate arguments before using model.')
            return -1
            
            
    def predict(self,X):
        '''Return the models predicted class for each of the given instances.'''
        try:
            return np.sign(self.decision_function(X))
        except AttributeError:
            print('Model not fitted, call \'fit\' with appropriate arguments before using model.')

In [5]:
def svm_train_dual ( data_train , label_train , regularisation_para_C ):
  svm_train_dual=SupportVectorMachine(C=regularisation_para_C)
  return svm_train_dual.fit(data_train , label_train)


##svm_train_dual=SupportVectorMachine(C=100)
#svm_train_dual.fit(data_train , label_train)
#svm_model_dual = svm_train_dual ( data_train , label_train , 100 )
svm_train_dual ( data_train , label_train , 100 )

     pcost       dcost       gap    pres   dres
 0: -1.8984e+04 -1.4320e+07  4e+07  8e-01  4e-11
 1: -1.0566e+04 -4.2388e+06  7e+06  1e-01  3e-11
 2: -4.7530e+03 -1.5794e+06  3e+06  4e-02  2e-11
 3: -3.8879e+02 -8.0859e+05  1e+06  2e-02  1e-11
 4:  2.5749e+03 -5.4135e+05  9e+05  1e-02  1e-11
 5:  8.7781e+03 -4.5896e+05  7e+05  6e-03  7e-12
 6:  8.3693e+03 -5.3356e+04  8e+04  5e-04  3e-12
 7:  2.5347e+03 -3.6284e+03  6e+03  9e-06  9e-13
 8:  3.5343e+02 -4.6458e+02  8e+02  8e-09  4e-13
 9:  8.8401e+01 -1.9841e+02  3e+02  2e-09  2e-13
10:  1.6644e+01 -1.0112e+02  1e+02  5e-10  1e-13
11:  1.9277e-01 -9.4623e+01  9e+01  3e-10  1e-13
12: -2.0754e+01 -7.0101e+01  5e+01  1e-10  2e-13
13: -3.0935e+01 -5.9294e+01  3e+01  3e-11  2e-13
14: -3.7930e+01 -4.8861e+01  1e+01  9e-12  2e-13
15: -4.0498e+01 -4.4827e+01  4e+00  9e-15  2e-13
16: -4.2193e+01 -4.2654e+01  5e-01  1e-15  2e-13
17: -4.2391e+01 -4.2405e+01  1e-02  3e-16  3e-13
18: -4.2397e+01 -4.2397e+01  3e-04  8e-15  3e-13
19: -4.2397e+01 -4.23